In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedKFold
from tqdm import tqdm
import warnings
warnings.filterwarnings("ignore")

In [2]:
data_folder = 'C:/Users/Николай/PycharmProjects/VKRecSys/data/'
train = pd.read_parquet(f'{data_folder}train_interactions.parquet', engine='pyarrow')
train['like'] = train['like'] + train['dislike'].replace({1: -1})
train.drop(columns=['dislike'], inplace=True)
train['like'] = train['like'].astype('int8')
train.rename(columns={'like' : 'target'}, inplace=True)

In [3]:
train = train.sort_values(by='user_id')

In [4]:
# Добавляем столбец для фолда
train['fold'] = -1

# Инициализируем tqdm
tqdm.pandas()

In [5]:
# Группируем по user_id
for user, group in tqdm(train.groupby('user_id'), desc="Processing users"):
    
    counts = group['target'].value_counts()
    
    # Проверка количества записей в группе
    if counts.max() < 4:
        print(1)
        # Если записей меньше 4, назначаем все в один фолд (например, 0)
        train.loc[group.index, 'fold'] = 0
    else:
        # Стратифицированная разбивка на 4 фолда
        skf = StratifiedKFold(n_splits=4, shuffle=True, random_state=42)
        
        # Получаем индексы для каждого фолда
        for fold, (train_index, test_index) in enumerate(skf.split(group, group['target'])):
            train.loc[group.index[test_index], 'fold'] = fold

Processing users:   5%|▌         | 9908/183404 [00:36<03:37, 796.82it/s] 

1


Processing users:  62%|██████▏   | 114463/183404 [03:05<01:20, 852.44it/s]

1


Processing users:  94%|█████████▍| 172727/183404 [04:24<00:12, 880.54it/s]

1


Processing users:  98%|█████████▊| 180439/183404 [04:33<00:03, 891.92it/s]

1


Processing users: 100%|██████████| 183404/183404 [04:36<00:00, 662.34it/s]


In [6]:
train[(train['user_id']==13)&(train['target']==0)]['fold'].value_counts()

1    58
0    58
2    58
3    57
Name: fold, dtype: int64

In [7]:
train[train['user_id']==114331]['target'].value_counts()

0    3
1    3
Name: target, dtype: int64

In [8]:
train['fold'].value_counts()

0    36485824
1    36439626
2    36393988
3    36347844
Name: fold, dtype: int64

In [9]:
series = train['fold']

In [10]:
series.to_csv('data/fold.csv', index=False)

In [11]:
series

34594896     2
134531857    0
113378626    1
49135032     1
19151932     0
            ..
138686636    1
112040078    1
112039761    1
145121692    0
18701593     3
Name: fold, Length: 145667282, dtype: int64